# Lecture 10

## Differential Equations II:

### From Words to Maths

In [ ]:
import numpy as np
import cmath
import sympy as sp
import scipy.integrate
sp.init_printing()

##################################################
##### Matplotlib boilerplate for consistency #####
##################################################
from ipywidgets import interact
from ipywidgets import FloatSlider
from ipywidgets import IntSlider
from matplotlib import pyplot as plt

%matplotlib widget

global_fig_width = 9
global_fig_height = global_fig_width / 1.61803399
font_size = 10

plt.rcParams['axes.axisbelow'] = True
plt.rcParams['axes.edgecolor'] = '0.8'
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.labelpad'] = 8
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['axes.titlepad'] = 16.0
plt.rcParams['axes.titlesize'] = font_size * 1.4
plt.rcParams['figure.figsize'] = (global_fig_width, global_fig_height)
plt.rcParams['font.sans-serif'] = ['Computer Modern Sans Serif', 'DejaVu Sans', 'sans-serif']
plt.rcParams['font.size'] = font_size
plt.rcParams['grid.color'] = '0.8'
plt.rcParams['grid.linestyle'] = 'dashed'
plt.rcParams['grid.linewidth'] = 2
plt.rcParams['lines.dash_capstyle'] = 'round'
plt.rcParams['lines.dashed_pattern'] = [1, 4]
plt.rcParams['xtick.labelsize'] = font_size
plt.rcParams['xtick.major.pad'] = 4
plt.rcParams['xtick.major.size'] = 0
plt.rcParams['ytick.labelsize'] = font_size
plt.rcParams['ytick.major.pad'] = 4
plt.rcParams['ytick.major.size'] = 0
##################################################

## Example of forming a differential equation from words:

The length $\;y\;$ cm of a leaf during the period of its growth is proportional to the amount of water it contains.
During this period the leaf retains a similar shape: that is, the ratio of its length to its width remains constant.
The leaf absorbs water from its plant at a rate proportional to $\;y\;$ and it loses water by evaporation at a rate proportional to the area
of the leaf at the time when its length is $\;y\;$ cm.

- Form a differential equation to describe the growth of the leaf.

1. Assume the length of the leaf is $\;y\;$ cm at time $\;t\;$ days after it was first observed.

2. Let the rate the leaf receives water be $\;k_1y\;$ where $\;k_1\;$ is a positive constant.

3. Area of leaf at time $\;t\;$ days is proportional to $\;y^2\;$ since it maintains its shape, so the leaf is losing water at a rate of $\;k_2y^2\;$ where $\;k_2\;$ is another positive constant. 

## Example of forming a differential equation from words:

1. Assume the length of the leaf is $\;y\;$ cm at time $\;t\;$ days after it was first observed.
2. Let the rate the leaf receives water be $\;k_1y\;$ where $\;k_1\;$ is a positive constant.
3. Area of leaf at time $\;t\;$ days is proportional to $\;y^2\;$ since it maintains its shape, so the leaf is losing water at a rate of $\;k_2y^2\;$ where $\;k_2\;$ is another positive constant. 


4. Rate of growth is given by $\displaystyle{ {\rm d}y\over {\rm d}t}$, which is the rate of change of its length.


$${{\rm d}y\over {\rm d}t}=k_1y-k_2y^2=y(k_1-k_2y)$$

```python
x = np.linspace(0,10,100)
def dydt(y,t):
    return k1*y - k2*y**2
y = scipy.integrate.odeint(dydt,0.1,x)
```

In [ ]:
class InteractiveODESolver2:
    def __init__(self):
        # Create the figure and axes
        self.fig, self.ax = plt.subplots()
        # Hide the figure header and footer
        self.fig.canvas.header_visible = False
        self.fig.canvas.footer_visible = False

        # Set labels and title
        self.ax.set_xlabel('$t$')
        self.ax.set_ylabel('$y$')
        self.ax.set_title(r'Solution of $\frac{dy}{dt} = k_1 y - k_2 y^2$')

        # Initialize x vector
        self.x = np.linspace(0, 10, 100)

        # Initial condition
        self.y0 = 0.1

        # Initial parameters k1 and k2
        self.k1 = 1.0
        self.k2 = 1.0

        # Solve the ODE with initial k1 and k2
        self.y = self.solve_ode(self.k1, self.k2)

        # Plot the initial solution
        self.line, = self.ax.plot(self.x, self.y, label=f'$k_1 = {self.k1:.2f}, k_2 = {self.k2:.2f}$')
        self.ax.legend()

    def solve_ode(self, k1, k2):
        # Define the ODE system
        def dydx(y, x):
            return k1 * y - k2 * y**2
        # Solve the ODE
        y = scipy.integrate.odeint(dydx, self.y0, self.x).flatten()
        return y

    def update(self, k1, k2):
        # Update parameters
        self.k1 = k1
        self.k2 = k2

        # Solve the ODE with new parameters
        self.y = self.solve_ode(self.k1, self.k2)

        # Update the line data
        self.line.set_data(self.x, self.y)

        # Update the label
        self.line.set_label(f'$k_1 = {self.k1:.2f}, k_2 = {self.k2:.2f}$')
        self.ax.legend()

        # Rescale the y-axis to fit the new data
        y_min, y_max = np.min(self.y), np.max(self.y)
        if y_min == y_max:
            y_min -= 0.1
            y_max += 0.1
        self.ax.set_ylim(y_min - 0.1 * abs(y_min), y_max + 0.1 * abs(y_max))

        # Redraw the figure canvas
        self.fig.canvas.draw_idle()

# Create the sliders
k1_slider = FloatSlider(value=1.0, min=0.01, max=2.0, step=0.01, continuous_update=True, description='$k_1$')
k2_slider = FloatSlider(value=1.0, min=0.01, max=2.0, step=0.01, continuous_update=True, description='$k_2$')

interactive_ode_solver2 = InteractiveODESolver2()
interact(interactive_ode_solver2.update, k1=k1_slider, k2=k2_slider);

## Example 2: Solid tumour growth.

An avascular tumour tends to grow in a spherical shape, depending on environmental restrictions.

Assuming that the growth rate of the tumour depends only on the availability of oxygen and nutrients, which enter the tumour solely by diffusing through its surface, derive a model for the change in tumour volume over time.

1. Denote the volume of the tumour at time $\;t\;$ by $\;V(t)\;$, and its radius by $\;R(t)$.


2. Since the tumour is a sphere, its volume $\;V\;$ is proportional to $\;R^3\;$ and its surface area to $\;R^2\;$, so the surface area is proportional to $\;V^{2/3}\;$.


3. The rate at which the tumour acquires nutrients, and hence the rate at which the volume increases, is thus proportional to $\;V^{2/3}$.


1. Denote the volume of the tumour at time $\;t\;$ by $\;V(t)\;$, and its radius by $\;R(t)$.
2. Since the tumour is a sphere, its volume $\;V\;$ is proportional to $\;R^3\;$ and its surface area to $\;R^2\;$, so the surface area is proportional to $\;V^{2/3}\;$.
3. The rate at which the tumour acquires nutrients, and hence the rate at which the volume increases, is thus proportional to $\;V^{2/3}$.


This gives us the equation:

$$\frac{{\rm d}V}{{\rm d}t} = kV^{2/3}$$

$$\frac{{\rm d}V}{{\rm d}t} = kV^{2/3}$$

Solve by separation of variables:

$$\int V^{-2/3}~{\rm d}V = \int k~{\rm d}t$$

$$V = \left({kt+c\over 3}\right)^3$$

where $c$ is a constant of integration, the value of which will depend upon the initial conditions.

In [ ]:
class InteractivePlot:
    def __init__(self):
        # Create the figure and axes
        self.fig, self.ax = plt.subplots()
        # Hide the figure header and footer
        self.fig.canvas.header_visible = False
        self.fig.canvas.footer_visible = False

        # Set labels and title
        self.ax.set_xlabel('$t$')
        self.ax.set_ylabel('$V$')
        self.ax.set_title('Plot of $V$ vs $t$')

        # Initialize t
        self.t = np.linspace(0, 10, 100)

        # Initial parameters k and c
        self.k = 1.0
        self.c = 10.0

        # Calculate initial v
        self.v = ((self.k * self.t + self.c) / 3) ** 3

        # Plot initial line
        self.line, = self.ax.plot(self.t, self.v, label=f'$k = {self.k:.2f}, c = {self.c:.2f}$')
        self.ax.legend()

    def update(self, k, c):
        # Update parameters
        self.k = k
        self.c = c

        # Recalculate v
        self.v = ((self.k * self.t + self.c) / 3) ** 3

        # Update the line data
        self.line.set_ydata(self.v)
        self.line.set_label(f'$k = {self.k:.2f}, c = {self.c:.2f}$')
        self.ax.legend()

        # Rescale y-axis to fit new data
        y_min, y_max = np.min(self.v), np.max(self.v)
        if y_min == y_max:
            y_min -= 0.1
            y_max += 0.1
        self.ax.set_ylim(y_min - 0.1 * abs(y_min), y_max + 0.1 * abs(y_max))

        # Redraw the canvas
        self.fig.canvas.draw_idle()

# Create the sliders
k_slider = FloatSlider(value=1.0, min=0.01, max=2.0, step=0.01, continuous_update=True, description='$k$')
c_slider = FloatSlider(value=10.0, min=0.01, max=20.0, step=0.01, continuous_update=True, description='$c$')

interactive_plot = InteractivePlot()
interact(interactive_plot.update, k=k_slider, c=c_slider);

## Second Order Differential Equations


**Example: Solve**  $\quad \displaystyle \frac{{\rm d}^2y}{{\rm d}x^2} = \left(\frac{{\rm d}y}{{\rm d}x}\right)^2$


We will use the substitution $\displaystyle z = \frac{{\rm d}y}{{\rm d}x}$.
This implies that $\displaystyle \frac{{\rm d}z}{{\rm d}x} = \frac{{\rm d}^2y}{{\rm d}x^2}$.


Substituting into the original equation, to eliminate $y$, gives

$$\frac{{\rm d}z}{{\rm d}x} = z^2$$

This is a separable equation:

$$\int z^{-2}~{\rm d}z = \int ~{\rm d}x \qquad\Rightarrow\qquad
  -{1\over z} = x + A \qquad\Rightarrow\qquad
  z = -{1\over x+A}$$

where $A$ is an arbitrary constant.



We can then replace $z$ using the original substitution to obtain a first order differential equation:
$$\frac{{\rm d}y}{{\rm d}x} = -{1\over x+A} ~~~~~~~~~~~~{\rm Equation~(9)}$$


This equation can be solved with a standard integral:

$$y = -\ln(x+A) + B$$

(how do we determine $A$ and $B$?)

Example

1. At $\;x=0,\;$ $\;\displaystyle \frac{{\rm d}y}{{\rm d}x} = -1\;$ and $\;y=0\;$.



We substitute the first condition into $\frac{{\rm d}y}{{\rm d}x} = -{1\over x+A}$ to obtain $\;A=1\;$.

Then substitute $A$ and the second condition into the eventual solution
to find $\;B=0$.

2. $\;y(0)=0\;$ and $\;y(e-1)=-1$.

This time both conditions can be substituted into the solution:

$$y(0)=0 \Rightarrow 0=B-\ln(A) \Rightarrow B=\ln(A)$$

$$y(e-1)=-1 \Rightarrow -1=\ln(A)-\ln{e-1+A} \Rightarrow A=1$$

## More tricks to do integration

Let's return to and solve our leaf example:

$${{\rm d}y\over {\rm d}t}=k_1y-k_2y^2=y(k_1-k_2y)$$


This is a separable differential equation: $$\;\displaystyle\int {{\rm d}y\over 
y(k_1-k_2y)} =\int {\rm d}t.\;$$

We can split apart the integral on the RHS using **partial fractions** in SymPy.

We want $\qquad\displaystyle{1\over y(k_1-k_2y)}={A\over y}+{B\over (k_1-k_2y)}$

```python
y, k1, k2 = sp.symbols('y k_1 k_2')
sp.apart(1 / (y*(k1 - k2*y)),y)
```

In [ ]:
y, k1, k2 = sp.symbols('y k_1 k_2')
sp.apart(1 / (y*(k1 - k2*y)),y)


So $A={1\over k_1}$ and $B={k_2\over k_1}$.

$${\rm~Thus,~~~~}\int {{\rm d}y\over y(k_1-k_2y)}=\int {{\rm d}y\over k_1y}+
\int {k_2\over k_1(k_1-k_2y)}{\rm d}y =\int dt$$


$${\rm~Thus~~~~~~ }{1\over k_1}\ln y-{k_2\over k_1k_2}\ln(k_1-k_2y)=t + c$$


We can get $c$ from initial conditions.
Try doing the algebraic manipulation of this to make $y$ the subject of the equation:

$$y={k_1\exp (k_1t+d)\over 1+ k_2 \exp(k_1t + d)}$$

where $d$ is a constant.